<p align="center">
  <img width='325' src="https://user-images.githubusercontent.com/8030363/161553611-51a40cf9-e348-4eff-91bd-ab03eac41dd3.png" />
</p>

***
***

# Knowledge Graph Entity Search Examples

***

**Author:** [TJCallahan](http://tiffanycallahan.com/)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Wiki Page:** [OWL-NETS-2.0](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0)  
**Release:** **[v3.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v3.0.0)**  
  
<br> 

## Purpose  
The goal of this notebook is to explore different ways to examine relationships between different types of entities in a PheKnowLator knowledge graph.

### Notebook Organization  
- [Set-Up Environment](#set-environment)  
- [Knowledge Graph Data](#kg-data)  
- [Knowledge-based Characterization](#kg-characterization)  
  - [Node-Level Characterization](#node-level)  
  - [Path-Level Characterization](#path-level)  

***
***

<br>

<br>

***  
## Set-Up Environment  <a class="anchor" id="set-environment"></a> 
***  
___

### Dependencies: [pkt_kg](https://pypi.org/project/pkt-kg/), [networkx](https://pypi.org/project/networkx/), [rdflib](https://pypi.org/project/rdflib/)

To prepare for the tutorial we need to make sure that the all needed libraries are downloaded and imported. If you don't already have `pkt_kg`, `rdflib`, and `networkx` installed, you can extend the code chunk below to include any libraries that you need to download. In addition to downloading needed libraries, you will also need to download the specific version of each knowledge graph that you want to analyze. Each data source is briefly described in the next section.  

In [ ]:
# # uncomment and run to install any required modules from notebooks/requirements.txt
# import sys
# !{sys.executable} -m pip install -r ../../notebooks/requirements.txt

In [ ]:
# # if running a local version (i.e., forked from GitHub) of pkt_kg, uncomment the code below
# import sys
# sys.path.append('../')

In [ ]:
# import needed libraries
import json
import networkx as nx
import os
import pandas as pd
import pickle
import random
import re

from pkt_kg.utils import *
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDFS
from tqdm.notebook import tqdm
from typing import Callable, Dict, List, Optional, Union

# create namespace for OBO ontologies
obo = Namespace('http://purl.obolibrary.org/obo/')

### Helper Functions 
Helper functions used only by this notebook that are needed to process and label knowledge graph node and edge entities.

In [ ]:
def format_path_ancestors(anc_dict: Dict, node_metadata: Dict) -> List:
    """Processes a dictionary of node ancestors into a list.

    Args:
        anc_dict: A dictionary where keys are ints formatted as strings and values are sets of URL strings for each
            concept that was found at that level. The level is the distance in the hierarchy from the searched node.
        node_metadata: A nested dictionary containing node attributes.

    Returns:
        ancestors: A nested list where each inner list contains ontology identifier strings.
    """

    ancestors = [['{} ({})'.format(node_metadata[str(x)]['label'], x) for x in anc_dict[str(k)]]
                 for k in sorted([int(x) for x in anc_dict.keys()])]

    return ancestors


def formats_node_information(node: URIRef, neighborhood: List, metadata_dict: Dict, verbose: bool=False) -> None:
    """Processes neighborhood results.
    
    Args:
        node: A string containing a node URL.
        neighborhood: A nested list of strings, where each string contains a node identifier.
        metadata_dict: A nested dictionary containing node attributes.
        verbose: A bool indicating whether or not node and edge metadata should be printed.
        
    Returns:
        None
    """
    
    for e, o in neighborhood:
        spe = '\n' if neighborhood.index([e, o]) == 0 else '\n\n'
        s, s_label = str(node[0]).split('/')[-1], metadata_dict[str(node[0])]['label']
        e_label = metadata_dict[str(e)]['label']
        o, o_label, o_def = str(o).split('/')[-1], metadata_dict[str(o)]['label'], metadata_dict[str(o)]['description']
        if verbose:
            if o_def != 'None': print(spe + '>>> {} ({}) - {} - {} ({})\n{} Definition: {}'.format(s_label, s, e_label, o_label, o, o, o_def))
            else: print(spe + '>>> {} ({}) - {} - {} ({})'.format(s_label, s, e_label, o_label, o))
        else: print('>>> {} ({}) - {} - {} ({})'.format(s_label, s, e_label, o_label, o))
    
    return None
    

def metadata_formatter(s: str, o: str, metadata_dict: Dict) -> None:
    """Function looks up edge-level metadata and prints it.
    
    Args:
        s: A string containing the identifier for the subject node of a predicate or triple.
        o: A string containing the identifier for the object node of a predicate or triple.
        metadata_dict: A nested dictionary containing node and edge-level metadata.
    
    Returns:
        None.
    """
    
    s = s + '-reactome_' if 'R-HSA' in s else s
    o = o + '-reactome_' if 'R-HSA' in o else o
    
    if s + '-' + o in metadata_dict['edges'].keys():
        print('\nEdge Evidence'); print(json.dumps(metadata_dict['edges'][s + '-' + o], indent=4))
    elif o + '-' + s in metadata_dict['edges'].keys():
         print('\nEdge Evidence'); print(json.dumps(metadata_dict['edges'][o + '-' + s], indent=4))
    else:
        pass
    
    return None


def formats_path_information(kg: nx.multidigraph.MultiDiGraph, paths: List, path_type: str, metadata_func: Callable, metadata_dict: Dict, node_metadata: Dict, verbose: bool=False, rand: bool=False, sample_size: int=10) -> None:
    """Processes shortest and simple path results.
    
    Args:
        kg: A networkx MultiDiGraph object.
        paths: A nested list of strings, where each string contains an an entity identifier.
        path_type: A string, either 'simple' or 'shortest' that indicates the types of paths to process.
        metadata_func: A function that processes edge metadata.
        metadata_dict: A nested dictionary containing node and edge-level metadata. 
        node_metadata: A nested dictionary containing node attributes.
        verbose: A bool indicating whether or not node and edge metadata should be printed.
        rand: A bool indicating whether or not to draw random samples from the path.
        sample_size: An integer used when rand is True to specify the size of the random sample to draw.
    
    Returns:
        None
    """
    
    if path_type == 'shortest': 
        for i in range(0, len(paths[0]) - 1):
            s = paths[0][i]; o = paths[0][i + 1]
            edges = kg.get_edge_data(*(s, o)).keys()
            for e in edges:
                spe = '\n' if list(edges).index(e) == 0 else '\n\n\n'
                s, s_label = str(s).split('/')[-1], node_metadata[str(s)]['label']
                e_label = node_metadata[str(e)]['label']
                o, o_label, o_def = str(o).split('/')[-1], node_metadata[str(o)]['label'], node_metadata[str(o)]['description']
                if verbose:
                    if o_def != 'None': print(spe + '>>> {} ({}) - {} - {} ({})\n\n{} Definition: {}'.format(s_label, s, e_label, o_label, o, o, o_def))
                    else: print(spe + '>>> {} ({}) - {} - {} ({})'.format(s_label, s, e_label, o_label, o))
                    metadata_func(s, o, metadata_dict)
                else: print('>>> {} ({}) - {} - {} ({})'.format(s_label, s, e_label, o_label, o))
    else:
        if rand: paths = random.sample(paths, sample_size)
        for path in paths:
            print('*' * 100)
            for i in range(0, len(path) - 1):
                spe = '\n' if i == 0 else '\n\n\n'
                s = path[i]; o = path[i + 1]; edges = kg.get_edge_data(*(s, o))
                try: edges.keys()
                except AttributeError: edges = kg.get_edge_data(*(o, s))
                for e in edges.keys():
                    s, s_label = str(s).split('/')[-1], node_metadata[str(s)]['label']
                    e_label = node_metadata[str(e)]['label']
                    o, o_label, o_def = str(o).split('/')[-1], node_metadata[str(o)]['label'], node_metadata[str(o)]['description']
                    if verbose:
                        if o_def != 'None': print(spe + '>>> {} ({}) - {} - {} ({})\n\n{} Definition: {}'.format(s_label, s, e_label, o_label, o, o, o_def))
                        else: print(spe + '>>> {} ({}) - {} - {} ({})'.format(s_label, s, e_label, o_label, o))
                        metadata_func(s, o, metadata_dict)
                    else: print('>>> {} ({}) - {} - {} ({})'.format(s_label, s, e_label, o_label, o))
            print('*' * 100); print('\n')
    
    return None
    

<br>

***

## Knowledge Graph Data  <a class="anchor" id="kg-data"></a>
***
___

This notebook was built using a `v3.0.2` OWL-NETS-abstracted subclass-based build with inverse relations, which is publicly available and can be downloaded using the following links:  
- [PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle](https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/inverse_relations/owlnets/PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle)  
- [PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NodeLabels.txt](https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/inverse_relations/owlnets/PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NodeLabels.txt)  


### Download Data  
***

The knowledge graph data is publicly available and downloaded from the PheKnowLator project's Google Cloud Storage Bucket: https://console.cloud.google.com/storage/browser/pheknowlator/. Data will be downloaded to a temporary directory created in the PheKnowLator root directory (`PheKnowLator/temp_directory`).

In [ ]:
# notebook will create a temporary directory and will download data to it
write_location = '../temp_directory/'
if not os.path.exists(write_location): os.mkdir(write_location)

In [ ]:
# download data to the data directory
data_urls = [
    'https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/inverse_relations/owlnets/PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle',
    'https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/inverse_relations/owlnets/PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NodeLabels.txt',
    'https://www.dropbox.com/s/ev0ea6v6fu70fbl/entity_metadata_dict.pkl?dl=1'
]

for url in data_urls:
    file_name = url.split('/')[-1] if 'entity_metadata_dict.pkl' not in url else re.sub(r'\?.*', '', url.split('/')[-1])
    if not os.path.exists(write_location + file_name): data_downloader(url, write_location, file_name)

### Loading Data
***

The knowledge graph will be loaded as a `networkx` MultiDiGraph object and the node labels will be read in and converted to a dictionary to enable easy access to node labels and other relevant metadata.

#### Knowledge Graph

In [ ]:
# load the knowledge graph
kg = nx.read_gpickle(write_location + data_urls[0].split('/')[-1])
print('The knowledge graph contains {} nodes and {} edges'.format(len(kg.nodes()), len(kg.edges())))

In [ ]:
# convert multidigraph to undirected graph -- needed to run some of the algorithms
undirected_kg = kg.to_undirected()

#### Load Node Metadata

In [ ]:
# read in node metadata
node_data = pd.read_csv(write_location + data_urls[1].split('/')[-1], header=0, sep=r"\t", encoding="utf8", engine='python', quoting=3)
node_data['entity_uri'] = node_data['entity_uri'].str.strip('<>')  # remove angle brackets
node_data.head()

In [ ]:
# convert node data to dictionary
node_data_dict = dict()
for idx, row in tqdm(node_data.iterrows(), total=node_data.shape[0]):
    node_data_dict[row['entity_uri']] = {
        'label': row['label'],
        'description': row['description/definition'],
        'synonym': row['synonym']
    }

#### Load Node and Edge Evidence
This file is temporary while the next release is being formatted.

In [ ]:
filepath = write_location + re.sub(r'\?.*', '', data_urls[2].split('/')[-1])
max_bytes = 2**31 - 1; input_size = os.path.getsize(filepath); bytes_in = bytearray(0)
with open(filepath, 'rb') as f_in:
    for _ in tqdm(range(0, input_size, max_bytes)):
        bytes_in += f_in.read(max_bytes)
metadata_dict = pickle.loads(bytes_in)

<br>

***

## Knowledge-based Characterization  <a class="anchor" id="kg-characterization"></a>
***
____

The goal is to use the knowledge graph to explore what we know about specific concepts as well as what we can say about pairs of concepts. Additional details are presented by comparison below:

#### [Node-Level](#node-level)
 - <u>Node Ancestry</u>: Identify all ancestors for each node up to the root.
 - <u>Node Neighborhood</u>: Returns all nodes reachable from a node of interest via a single directed edge.   


#### [Path-Level](#path-level)
  - <u>All Shortest Paths</u>: Returns the shortest simple path, if there are multiple paths of the same length then they are all returned.
  - <u>All Simple Paths</u>: A simple path is a path with no repeated nodes. These nodes are identified using a modified depth-first search. Given that there are a lot of these, the initial output is limited to a random draw of 10 paths of length 10 from the first 100 derived paths.
  
<br>

**Important.** Output for the node neighborhood and simple and shortest paths are printed twice. The first time (`verbose=False`), there is minimal node and edge evidence printed. The second time (`verbose=True`), node definitions and any available evidence from the source resources used to build the edge are printed. Note that the metadata for the edges in the neighborhood will only include a definition for the nodes that are connected to each primary node of interest.

---

<br>

### Node-Level Characterization  <a class="anchor" id="node-level"></a>
***

This section characterizes the following concepts:
- [benazepril (`CHEBI_3011`)](#chebi1)  
- [hydrochlorothiazide (`CHEBI_5778`)](#chebi2)  
- [Acute Myocardial Infarction (`MONDO_0004781`)](#mondo1)  
- [Myocardial infarction (`HP_0001658`)](#hpo1)

*Note*. All output is presented twice for each analysis, the first without any metadata/evidence and the second time, with metadata. This is done to facilitate readability.

#### benazepril (`CHEBI_3011`) <a class="anchor" id="chebi1"></a>

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'CHEBI'; node = [obo.CHEBI_3011]
chebi3011_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
chebi3011_ancestors = format_path_ancestors(chebi3011_anc_dict, node_data_dict)

# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}\n'.format(node[0]))
for level in range(len(chebi3011_ancestors)):
    print('Level: {}'.format(str(level + 1)))
    for v in chebi3011_ancestors[level]:
        print('\t- {}'.format(re.sub('http://purl.obolibrary.org/obo/', '', v)))

*Neighborhood*

In [ ]:
# examine the node's neigborhood
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
chebi3011_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

In [ ]:
# print nodes without definitions
formats_node_information(node, chebi3011_sorted_neigbors, node_data_dict, verbose=False)

In [ ]:
# print nodes with definitions
formats_node_information(node, chebi3011_sorted_neigbors, node_data_dict, verbose=True)

<br>

**hydrochlorothiazide (`CHEBI_5778`)** <a class="anchor" id="chebi2"></a>

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'CHEBI'; node = [obo.CHEBI_5778]
path_list = nx_ancestor_search(kg, node.copy(), prefix)
chebi5778_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
chebi5778_ancestors = format_path_ancestors(chebi5778_anc_dict, node_data_dict)

# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}\n'.format(node[0]))
for level in range(len(chebi5778_ancestors)):
    print('Level: {}'.format(str(level + 1)))
    for v in chebi5778_ancestors[level]:
        print('\t- {}'.format(re.sub('http://purl.obolibrary.org/obo/', '', v)))

*Neighborhood*

In [ ]:
# examine the node's neigborhood
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
chebi5778_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

In [ ]:
# print nodes without definitions
formats_node_information(node, chebi5778_sorted_neigbors, node_data_dict, verbose=False)

In [ ]:
# print nodes with definitions
formats_node_information(node, chebi5778_sorted_neigbors, node_data_dict, verbose=True)

<br>

**Myocardial Infarction (`MONDO_0005068`)** <a class="anchor" id="mondo1"></a>

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'MONDO'; node = [obo.MONDO_0005068]
path_list = nx_ancestor_search(kg, node.copy(), prefix)
mondo0005068_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
mondo0005068_ancestors = format_path_ancestors(mondo0005068_anc_dict, node_data_dict)

# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}\n'.format(node[0]))
for level in range(len(mondo0005068_ancestors)):
    print('Level: {}'.format(str(level + 1)))
    for v in mondo0005068_ancestors[level]:
        print('\t- {}'.format(re.sub('http://purl.obolibrary.org/obo/', '', v)))

*Neighborhood*

In [ ]:
# examine the node's neigborhood
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
mondo0005068_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

In [ ]:
# print nodes without definitions
formats_node_information(node, mondo0005068_sorted_neigbors, node_data_dict, verbose=False)

In [ ]:
# print nodes with definitions
formats_node_information(node, mondo0005068_sorted_neigbors, node_data_dict, verbose=True)

<br>

**Myocardial infarction (`HP_0001658`)** <a class="anchor" id="hpo1"></a>

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'HP'; node = [obo.HP_0001658]
path_list = nx_ancestor_search(kg, node.copy(), prefix)
hp0001658_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
hp0001658_ancestors = format_path_ancestors(hp0001658_anc_dict, node_data_dict)

# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}\n'.format(node[0]))
for level in range(len(hp0001658_ancestors)):
    print('Level: {}'.format(str(level + 1)))
    for v in hp0001658_ancestors[level]:
        print('\t- {}'.format(re.sub('http://purl.obolibrary.org/obo/', '', v)))

*Neighborhood*

In [ ]:
# examine the node's neigborhood
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
hp0001658_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

In [ ]:
# print nodes without definitions
formats_node_information(node, hp0001658_sorted_neigbors, node_data_dict, verbose=False)

In [ ]:
# print nodes with definitions
formats_node_information(node, hp0001658_sorted_neigbors, node_data_dict, verbose=True)

<br>

### Path-Level Characterization  <a class="anchor" id="path-level"></a>

***

This section characterizes the following concept pairs:
- [benazepril (`CHEBI_3011`) - Myocardial Infarction (`MONDO_0005068`)](#pair1)     
- [hydrochlorothiazide (`CHEBI_5778`) - Myocardial Infarction (`MONDO_0005068`)](#pair2)  
- [benazepril (`CHEBI_3011`) - Myocardial infarction (`HP_0001658`)](#pair3)     
- [hydrochlorothiazide (`CHEBI_5778`) - Myocardial infarction (`HP_0001658`)](#pair4)  

*Note*. All output is presented twice for each analysis, the first without any metadata/evidence and the second time, with metadata. This is done to facilitate readability.

**benazepril (`CHEBI_3011`) - Myocardial Infarction (`MONDO_0005068`)** <a class="anchor" id="pair1"></a>

*Shortest Paths*

In [ ]:
# look at all shortest paths between the nodes in pair
shortest_paths = list(nx.all_shortest_paths(kg, obo.CHEBI_3011, obo.MONDO_0005068))
formats_path_information(kg=kg,
                         paths=shortest_paths,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True)

*Simple Paths*

In [ ]:
# look at all simple paths between the nodes in pair
simple_paths = []; counter = 0
for path in tqdm(nx.all_simple_paths(undirected_kg, source=obo.CHEBI_3011, target=obo.MONDO_0005068, cutoff=10)):
    simple_paths += [path]
    if counter == 100: break
    else: counter += 1

In [ ]:
# print path information -- without definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

In [ ]:
# print path information -- with definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True,
                         rand=True,
                         sample_size=10)

<br>

**hydrochlorothiazide (`CHEBI_5778`) - Myocardial Infarction (`MONDO_0005068`)** <a class="anchor" id="pair2"></a>

*Shortest Paths*

In [ ]:
# look at all shortest paths between the nodes in pair
shortest_paths = list(nx.all_shortest_paths(kg, obo.CHEBI_5778, obo.MONDO_0005068))
formats_path_information(kg=kg,
                         paths=shortest_paths,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True)

*Simple Paths*

In [ ]:
# look at all simple paths between the nodes in pair
simple_paths = []; counter = 0
for path in tqdm(nx.all_simple_paths(undirected_kg, source=obo.CHEBI_5778, target=obo.MONDO_0005068, cutoff=10)):
    simple_paths += [path]
    if counter == 100: break
    else: counter += 1

In [ ]:
# print path information -- without definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

In [ ]:
# print path information -- with definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True,
                         rand=True,
                         sample_size=10)

<br>

**benazepril (`CHEBI_3011`) - Myocardial infarction (`HP_0001658`)** <a class="anchor" id="pair3"></a>

*Shortest Paths*

In [ ]:
# look at all shortest paths between the nodes in pair
shortest_paths = list(nx.all_shortest_paths(kg, obo.CHEBI_3011, obo.HP_0001658))
formats_path_information(kg=kg,
                         paths=shortest_paths,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True)

*Simple Paths*

In [ ]:
# look at all simple paths between the nodes in pair
simple_paths = []; counter = 0
for path in tqdm(nx.all_simple_paths(undirected_kg, source=obo.CHEBI_3011, target=obo.HP_0001658, cutoff=10)):
    simple_paths += [path]
    if counter == 100: break
    else: counter += 1

In [ ]:
# print path information -- without definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

In [ ]:
# print path information -- with definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True,
                         rand=True,
                         sample_size=10)

<br>

**hydrochlorothiazide (`CHEBI_5778`) - Myocardial infarction (`HP_0001658`)** <a class="anchor" id="pair4"></a>

*Shortest Paths*

In [ ]:
# look at all shortest paths between the nodes in pair
shortest_paths = list(nx.all_shortest_paths(kg, obo.CHEBI_5778, obo.HP_0001658))
formats_path_information(kg=kg,
                         paths=shortest_paths,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict, 
                         verbose=True)

*Simple Paths*

In [ ]:
# look at all simple paths between the nodes in pair
simple_paths = []; counter = 0
for path in tqdm(nx.all_simple_paths(undirected_kg, source=obo.CHEBI_5778, target=obo.HP_0001658, cutoff=10)):
    simple_paths += [path]
    if counter == 100: break
    else: counter += 1

In [ ]:
# print path information -- without definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

In [ ]:
# print path information -- with definitions and metadata
formats_path_information(kg=kg,
                         paths=simple_paths,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=metadata_dict,
                         node_metadata=node_data_dict,
                         verbose=True,
                         rand=True,
                         sample_size=10)


<br>

***
***

This Notebook is part of the [**PheKnowLator Ecosystem**](https://zenodo.org/communities/pheknowlator-ecosystem/edit/)

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```

***